In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, os.path

dataset = '/home/jupyter/data_mount/compressed_dataset'   # mounted folder (mounted GCP bucket containing folder)
# dataset = '/content/drive/Shareddrives/229 proj/compressed_dataset'
# print(os.listdir(dataset))
dict = {}
for country in os.listdir(dataset):
    f_country = dataset+'/'+country
    if os.path.isdir(f_country):
        n_pics = len(os.listdir(f_country))
        if n_pics > 150:
            dict[n_pics] = country
A = []
for key in sorted(dict.keys()):
    A.append(f"#pics: {key} = {dict[key]}")
# max_n_pics = len(A)
max_n_pics = 10
for i in range(-1,-max_n_pics,-1):
    print(f"{i}: {A[i]}")
    
# hello

-1: #pics: 12014 = United States
-2: #pics: 3840 = Japan
-3: #pics: 3573 = France
-4: #pics: 2484 = United Kingdom
-5: #pics: 2320 = Brazil
-6: #pics: 1761 = Russia
-7: #pics: 1704 = Australia
-8: #pics: 1382 = Canada
-9: #pics: 1183 = South Africa


In [ ]:
# import resnet50 model and weights 
import torch
from torchvision import datasets, models, transforms
from torchvision.models import resnet50, ResNet50_Weights, alexnet, AlexNet_Weights, squeezenet1_0, SqueezeNet1_0_Weights, vgg16, VGG16_Weights, densenet201, DenseNet201_Weights, inception_v3, Inception_V3_Weights
from torchvision.datasets import ImageFolder
import torch.optim as optim
from torch import nn
import time
import copy

In [ ]:

############# Baseline Section ############### 
# Feature extraction (not finetuning). 

# Select the most up-to-date weights (default)
model_resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
model_alexnet = alexnet(weights=AlexNet_Weights.DEFAULT)
model_squeezenet = squeezenet1_0(weights=SqueezeNet1_0_Weights.DEFAULT)
model_vgg = vgg16(weights=VGG16_Weights.DEFAULT)
model_densenet = densenet201(weights=DenseNet201_Weights.DEFAULT)
model_inception = inception_v3(weights=Inception_V3_Weights.DEFAULT)

# Initialize the Weight Transforms
weights_resnet = ResNet50_Weights.DEFAULT
weights_alexnet = AlexNet_Weights.DEFAULT
weights_squeezenet = SqueezeNet1_0_Weights.DEFAULT
weights_vgg = VGG16_Weights.DEFAULT
weights_densenet = DenseNet201_Weights.DEFAULT
weights_inception = DenseNet201_Weights.DEFAULT


preprocess_resnet = weights_resnet.transforms() 
preprocess_alexnet = weights_alexnet.transforms() 
preprocess_squeezenet = weights_squeezenet.transforms() 
preprocess_vgg = weights_vgg.transforms() 
preprocess_densenet = weights_densenet.transforms() 
preprocess_inception = weights_inception.transforms() 
# run the images through this preprocess()
# e.g: batch = preprocess(img).unsqueeze(0)

# data_path = 'drive/Shareddrives/229 proj/compressed_dataset'

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_names = ["resnet", "alexnet", "vgg", "squeezenet", "densenet", "inception"]

# Number of classes in the dataset
num_classes = 124

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Number of epochs to train for
num_epochs = 16

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth


...

  0%|          | 0.00/528M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth


  0%|          | 0.00/77.4M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

In [ ]:
# if feature_extracting is on, we don't want to update the weights in the network
# except for the last layer. This freezes function the model params. 
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
set_parameter_requires_grad(model_resnet, feature_extract)
set_parameter_requires_grad(model_alexnet, feature_extract)
set_parameter_requires_grad(model_squeezenet, feature_extract)
set_parameter_requires_grad(model_vgg, feature_extract)
set_parameter_requires_grad(model_densenet, feature_extract)
set_parameter_requires_grad(model_inception, feature_extract)

In [ ]:
# model training code 
def train_model(model, dataloaders, criterion, optimizer, num_epochs, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                print(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()
      
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
# Data and model initialization 
data_path = 'drive/Shareddrives/229 proj/compressed_dataset'

# modify fc layer so output matches our num_classes

num_resnet = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_resnet, num_classes)
model_alexnet.classifier[6] = nn.Linear(4096, num_classes)
model_vgg.classifier[6] = nn.Linear(4096, num_classes)
model_squeezenet.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
model_densenet.classifier = nn.Linear(1024, num_classes)
model_inception.AuxLogits.fc = nn.Linear(768, num_classes)
model_inception.fc = nn.Linear(2048, num_classes)


# print(model)

# Resnet50 takes inputs of dim (224,224,3)
input_size = 224

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #idk what to hardcode these to
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #idk what to hardcode these to
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_path), data_transforms[x]) for x in ['train', 'val']}



# print(image_datasets['train'])
# print(preprocess_resnet)

# print(image_datasets)

# Next step:
# Define a PyTorch data loader for the images
# data_loader = torch.utils.data.DataLoader(
#     ImageFolder(img_dir, transform=transforms.ToTensor()),
#     batch_size=batch_size,
#     shuffle=True
# )

dataloaders_dict = {
    x: torch.utils.data.DataLoader(image_datasets[x], 
                                   batch_size=batch_size, 
                                   shuffle=True, 
                                   num_workers=2
                                  ) 
    for x in ['train', 'val']
}




In [ ]:
# Get all the countries
sorted_countries = enumerate(sorted([country for country in os.listdir(data_path) if '.' not in country]))

countries = {i: (country, len(sorted_countries[i])) for i, country in sorted_countries}
print(countries)

{'Aland': 0, 'Albania': 1, 'American Samoa': 2, 'Andorra': 3, 'Antarctica': 4, 'Argentina': 5, 'Armenia': 6, 'Australia': 7, 'Austria': 8, 'Bangladesh': 9, 'Belarus': 10, 'Belgium': 11, 'Bermuda': 12, 'Bhutan': 13, 'Bolivia': 14, 'Botswana': 15, 'Brazil': 16, 'Bulgaria': 17, 'Cambodia': 18, 'Canada': 19, 'Chile': 20, 'China': 21, 'Colombia': 22, 'Costa Rica': 23, 'Croatia': 24, 'Curacao': 25, 'Czechia': 26, 'Denmark': 27, 'Dominican Republic': 28, 'Ecuador': 29, 'Egypt': 30, 'Estonia': 31, 'Eswatini': 32, 'Faroe Islands': 33, 'Finland': 34, 'France': 35, 'Germany': 36, 'Ghana': 37, 'Gibraltar': 38, 'Greece': 39, 'Greenland': 40, 'Guam': 41, 'Guatemala': 42, 'Hong Kong': 43, 'Hungary': 44, 'Iceland': 45, 'India': 46, 'Indonesia': 47, 'Iraq': 48, 'Ireland': 49, 'Isle of Man': 50, 'Israel': 51, 'Italy': 52, 'Japan': 53, 'Jersey': 54, 'Jordan': 55, 'Kenya': 56, 'Kyrgyzstan': 57, 'Laos': 58, 'Latvia': 59, 'Lebanon': 60, 'Lesotho': 61, 'Lithuania': 62, 'Luxembourg': 63, 'Macao': 64, 'Madagas

In [ ]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Send the model to GPU / CPU
model_resnet = model_resnet.to(device)

In [ ]:

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_resnet.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_resnet.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_resnet.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Optimize the params in params_to_update using SGD
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train  model
model, hist = train_model(model_resnet, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)



Epoch 0/15
----------
tensor([ 16,  97, 120,  35,  81, 120,  35,  19,  52,  19, 120,  88, 105,  71,
         35, 120,  47,  83,  35,  53,  34,  36,  35, 120,  53,  97,  42,  24,
         81, 120, 120,  94], device='cuda:0')
tensor([119,  19,  35,  16,  35, 120, 120,  16,  76,   5, 120, 101, 120,  88,
         98, 120, 120, 120, 108, 110, 114, 120, 103, 101,  81,   7,  94, 120,
         83, 120,  79,  51], device='cuda:0')
tensor([120, 119,  35, 120, 120, 120, 114,  16, 120, 120,  34,  47, 120,   7,
         31,  53,   7, 120, 119,   5, 120, 110, 112, 120,   5, 120, 120, 110,
         53, 120,   7, 120], device='cuda:0')
tensor([ 35, 120,  53,  98,  53,  53, 120,  66,  77,  39,  88,  53,  94,  34,
         35,  36,  19, 120,  53,  35, 120, 103,  19,  35,  35, 120, 120,  86,
        105,   8,  76,  16], device='cuda:0')
tensor([ 35,  76,  16,  53, 120,  35,  35,  53,  19, 120,   8,  77, 120,  53,
         53,  19,  53, 120,  88, 119,  61,  53, 119, 120, 110,  36,  35, 112,
         16,  

In [ ]:
######### Baseline Section End ############################################# 


In [ ]:
url = "../input/geolocation-geoguessr-images-50k/compressed_dataset/Reunion/"
!ls
!cat sample_data/README.md


In [ ]:
# chat-GPT filler code:
# Import the necessary libraries
import torch
import torchvision
from torch.utils.data import random_split
from torch.optim import SGD

# Load the pre-trained ResNet model
model = torchvision.models.resnet18(pretrained=True)

# Set the model to evaluation mode
model.eval()

# Load your own data
data = ...

# Split the data into a train set and a test set
train_data, test_data = random_split(data, [90, 10])

# Define the cross entropy loss function
criterion = torch.nn.CrossEntropyLoss()

# Define the SGD optimizer
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
for input, target in train_data:
    # Forward pass
    output = model(input)
    loss = criterion(output, target)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()

    # Update the weights
    optimizer.step()

# Test the model
with torch.no_grad():
    for input, target in test_data:
        # Forward pass
        output = model(input)

        # Compute the loss
        loss = criterion(output, target)

        # Print the loss
        print(loss)
